In [1]:
import urllib

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')
submission = pd.read_csv("data/sample_submission.csv")

In [5]:
train_data = train_data.mask(train_data.astype(object).eq('None')).dropna()
train_data.landmark_id = pd.to_numeric(train_data.landmark_id, errors='coerce')

test_data = test_data.mask(test_data.astype(object).eq('None')).dropna()

submission = submission.mask(submission.astype(object).eq('None')).dropna()

In [10]:
for index, row in train_data.iterrows():
    f = open("data/train/"+row['id']+".jpg",'wb')
    f.write(urllib.urlopen(row['url']).read())
    f.close()
    #urllib.urlretrieve(row['url'], "data/train/"+row['id']+".jpg")

KeyboardInterrupt: 

In [ ]:
for index, row in test_data.iterrows():
    urllib.urlretrieve(row['url'], "data/test/"+row['id']+".jpg")